In [3]:
import os
import pathlib
import datasets.deep_fashion

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from torch.utils.data import DataLoader, Subset
from torchvision.transforms import Resize, Lambda, Compose

import torch
import torchvision

import utils

from tqdm.notebook import tqdm

In [4]:
capbm_image_transform = torchvision.models.ResNet50_Weights.DEFAULT.transforms()
capbm_image_transform.antialias = True

capbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Category and Attribute Prediction Benchmark")
capbm_dataset = datasets.deep_fashion.CatAttrPredBM(capbm_dataset_dir, capbm_image_transform)

capbm_train_dataset = Subset(capbm_dataset, capbm_dataset.get_split_mask_idxs("train"))
capbm_test_dataset = Subset(capbm_dataset, capbm_dataset.get_split_mask_idxs("test"))
capbm_val_dataset = Subset(capbm_dataset, capbm_dataset.get_split_mask_idxs("val"))

###

ctsrbm_image_transform = torchvision.models.ResNet50_Weights.DEFAULT.transforms()
ctsrbm_image_transform.antialias = True

ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = datasets.deep_fashion.ConsToShopClothRetrBM(ctsrbm_dataset_dir, ctsrbm_image_transform)

ctsrbm_train_dataset = Subset(ctsrbm_dataset, ctsrbm_dataset.get_split_mask_idxs("train"))
ctsrbm_test_dataset = Subset(ctsrbm_dataset, ctsrbm_dataset.get_split_mask_idxs("test"))
ctsrbm_val_dataset = Subset(ctsrbm_dataset, ctsrbm_dataset.get_split_mask_idxs("val"))

---

In [5]:
import datasets.combine

In [6]:
fused_dataset = datasets.combine.Fusion([
    capbm_train_dataset,
    ctsrbm_train_dataset
])

In [7]:
len(fused_dataset)

308054

In [23]:
idx = 1
dataset_idx = fused_dataset[idx][0]

print("Dataset idx: {:d}".format(dataset_idx))

if dataset_idx == 0:

    print("Classification")
    print(fused_dataset[idx][1][0].shape)
    print(fused_dataset[idx][1][1])

if dataset_idx == 1:

    print("Retrieval")
    print(fused_dataset[idx][1][0].shape)
    print(fused_dataset[idx][1][1].shape)
    print(fused_dataset[idx][1][2].shape)

Dataset idx: 1
Retrieval
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


---

In [61]:
dataset_items = None

In [119]:
def custom_collate(original_batch):

    new_batch = []

    dataset_idxs = np.asarray([item[0] for item in original_batch])
    unique_dataset_idxs = np.unique(dataset_idxs)

    for dataset_idx in unique_dataset_idxs:

        dataset_batch = []

        dataset_items = [item[1] for item in original_batch if item[0] == dataset_idx]

        for subitem_idx in range(len(dataset_items[0])):

            dataset_subitems = [item[subitem_idx] for item in dataset_items]
            if type(dataset_subitems[0]) == int:
                dataset_subitems = list(map(torch.tensor, dataset_subitems))

            dataset_batch.append(torch.stack(dataset_subitems))

        new_batch.append((dataset_idx, dataset_batch))

    return new_batch

In [120]:
fused_data_loader = DataLoader(fused_dataset, batch_size=8, collate_fn=custom_collate)

In [129]:
for batch in fused_data_loader:

    print(type(batch))
    print(len(batch))

    print(batch[0][0])
    print(batch[0][1][0].shape)
    print(batch[0][1][1].shape)
    print(batch[0][1][2].shape)

    print(batch[1][0])
    print(batch[1][1][0].shape)
    print(batch[1][1][1].shape)
    print(batch[1][1][2].shape)
    print(batch[1][1][3].shape)

    break

<class 'list'>
2
0
torch.Size([5, 3, 224, 224])
torch.Size([5])
torch.Size([5, 1000])
1
torch.Size([3, 3, 224, 224])
torch.Size([3, 3, 224, 224])
torch.Size([3, 3, 224, 224])
torch.Size([3, 303])


In [65]:
dataset_items

---

In [81]:
original_batch = [
    (0, capbm_train_dataset[0]),
    (1, ctsrbm_train_dataset[0]),
    (0, capbm_train_dataset[1]),
    (0, capbm_train_dataset[2]),
    (1, ctsrbm_train_dataset[1]),
    (0, capbm_train_dataset[3]),
    (0, capbm_train_dataset[4]),
    (1, ctsrbm_train_dataset[2])
]

In [105]:
dataset_idxs = np.asarray([item[0] for item in original_batch])
unique_dataset_idxs = np.unique(dataset_idxs)

new_batch = []

for dataset_idx in unique_dataset_idxs:

    dataset_items = [item[1] for item in original_batch if item[0] == dataset_idx]

    num_items = 

    dataset_batch = [torch.stack([torch.tensor(item[idx]) for item in dataset_items]) for idx in range(len(dataset_items[0]))]

    new_batch.append((dataset_idx, dataset_batch))

/tmp/ipykernel_2929807/2388173343.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_batch = [torch.stack([torch.tensor(item[idx]) for item in dataset_items]) for idx in range(len(dataset_items[0]))]


In [96]:
torch.stack([item[0] for item in dataset_items]).shape

torch.Size([5, 3, 224, 224])

In [118]:
type(capbm_train_dataset[0][0]) == int

False